In [52]:
#STILL WIP

# # Read from:
# Dune - OP Deployed by deployer address type 
# Defillama/Subgraphs - TVL Flows by Program
# Notion - OP Budget by Program

# Join these datasets together on program & associate anything else to the generalized programs

In [53]:
import pandas as pd
import numpy as np

In [54]:
tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
distrib_df = pd.read_csv('csv_outputs/dune_op_distribution_type.csv')
program_df = pd.read_csv('inputs/OP Summer Programs [Public].csv')

In [55]:
#Filter TVL DF
tvl = tvl[tvl['include_in_summary'] == 1]
tvl['join_key'] = tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')
# display(tvl)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_23367/3179265136.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tvl['join_key'] = tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')


In [56]:
#Set up Distributions for Mapping
distrib_df['program_map'] = np.where(
                                distrib_df['to_type'].isin(tvl['top_level_name'])
                                ,distrib_df['to_type']
                                ,''
                                )

sum_distrib_df = distrib_df[['from_name','op_deployed','to_label','to_type','program_map']].groupby(['from_name','program_map']).sum()
sum_distrib_df.reset_index(inplace=True)
#Joins should maybe just be the program map OR from name, since Velo operated bribes for a while
sum_distrib_df['join_key'] = sum_distrib_df['from_name'] + ' - ' \
                                + np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_23367/3745196205.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum_distrib_df = distrib_df[['from_name','op_deployed','to_label','to_type','program_map']].groupby(['from_name','program_map']).sum()


In [59]:
#lowercase joinkeys
tvl['join_key'] = tvl['join_key'].str.lower()
sum_distrib_df['join_key'] = sum_distrib_df['join_key'].str.lower()
#
joins = sum_distrib_df.merge(tvl, on='join_key',how='outer')
display(joins[joins['join_key'].str.contains('velodr')])

,from_name,program_map,op_deployed,join_key,date,include_in_summary,top_level_name,program_name,app_name,num_op,...,period,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,flows_retention,last_price_net_dollar_flows_retention
3,Alchemix,alETH/WETH: Velodrome,10000.0,alchemix - aleth/weth: velodrome,2023-03-10,1.0,alETH/WETH: Velodrome,Alchemix - alETH/WETH: Velodrome,Alchemix,NaN,...,During Program,Gov Fund - Season 1,2023-01-08,NaN,3.541272e+06,3.541272e+06,3.533613e+06,3.533613e+06,NaN,NaN
5,Angle,OP/agEUR: Velodrome,3400.0,angle - op/ageur: velodrome,2023-03-10,1.0,OP/agEUR: Velodrome,Angle - OP/agEUR: Velodrome,Angle,NaN,...,During Program,Gov Fund - Season 1,2023-02-20,NaN,5.648418e+05,5.648418e+05,5.570202e+05,5.570202e+05,NaN,NaN
10,Beefy Finance,BIFI/OP: Velodrome,41000.0,beefy finance - bifi/op: velodrome,2023-03-10,1.0,BIFI/OP: Velodrome,Beefy Finance - BIFI/OP: Velodrome,Beefy Finance,NaN,...,During Program,Gov Fund - Phase 0,2022-09-13,NaN,4.444872e+05,4.444872e+05,5.123365e+05,5.123365e+05,NaN,NaN
11,Beefy Finance,BIFI/WETH: Velodrome,68875.0,beefy finance - bifi/weth: velodrome,2023-03-10,1.0,BIFI/WETH: Velodrome,Beefy Finance - BIFI/WETH: Velodrome,Beefy Finance,NaN,...,During Program,Gov Fund - Phase 0,2022-09-13,NaN,1.942103e+06,1.942103e+06,1.976264e+06,1.976264e+06,NaN,NaN
33,Layer2dao,L2DAO/OP: Velodrome,14000.0,layer2dao - l2dao/op: velodrome,2022-09-21,1.0,L2DAO/OP: Velodrome,Layer2Dao - L2DAO/OP: Velodrome,Layer2Dao,NaN,...,Post-Program,Gov Fund - Phase 0,2022-07-20,2022-08-22,4.681078e+05,1.750505e+05,3.585146e+05,1.175015e+05,0.373953,0.327745
43,Overnight,USD+/DOLA: Velodrome,8500.0,overnight - usd+/dola: velodrome,2023-03-10,1.0,USD+/DOLA: Velodrome,Overnight - USD+/DOLA: Velodrome,Overnight,NaN,...,During Program,Gov Fund - Season 1,2023-01-30,NaN,4.024876e+06,4.024876e+06,4.084762e+06,4.084762e+06,NaN,NaN
44,Overnight,USD+/USDC: Velodrome,21700.0,overnight - usd+/usdc: velodrome,2023-03-10,1.0,USD+/USDC: Velodrome,Overnight - USD+/USDC: Velodrome,Overnight,NaN,...,During Program,Gov Fund - Season 1,2023-01-10,NaN,1.656980e+06,1.656980e+06,2.053944e+06,2.053944e+06,NaN,NaN
51,QiDao,sUSDC/MAI: Velodrome,195333.0,qidao - susdc/mai: velodrome,2023-03-10,1.0,sUSDC/MAI: Velodrome,Qidao - sUSDC/MAI: Velodrome,Qidao,NaN,...,During Program,Gov Fund - Season 1,2022-08-26,NaN,2.668667e+06,2.668667e+06,2.877288e+06,2.877288e+06,NaN,NaN
52,QiDao,vUSDC/MAI: Velodrome,6000.0,qidao - vusdc/mai: velodrome,2023-03-10,1.0,vUSDC/MAI: Velodrome,Qidao - vUSDC/MAI: Velodrome,Qidao,NaN,...,During Program,Gov Fund - Season 1,2022-08-26,NaN,2.492702e+05,2.492702e+05,2.493918e+05,2.493918e+05,NaN,NaN
58,Rocket Pool,sWETH/rETH: Velodrome,21430.0,rocket pool - sweth/reth: velodrome,2023-03-10,1.0,sWETH/rETH: Velodrome,Rocket Pool - sWETH/rETH: Velodrome,Rocket Pool,NaN,...,Post-Program,Gov Fund - Season 1,2022-11-09,2023-02-08,2.988968e+06,-2.760986e+05,2.906730e+06,-3.677343e+05,0.092373,0.126511
